In [2]:
!pip install numpy

In [3]:
# imports
import requests
import json
import pandas as pd
import os
import time
from dotenv import load_dotenv

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "C:\Users\Goerge\anaconda3\envs\perm_enviro\python.exe"
  * The NumPy version is: "1.21.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed: The specified module could not be found.


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice.

In [ ]:
data = pd.read_csv('df_city_bike.csv')

In [ ]:
data.dropna(subset=['latitude', 'longitude'], inplace=True)
# creation of variable with lon and lat together
data['ll'] = data['latitude'].astype(str) + ',' + data['longitude'].astype(str)
data = data[data['ll'] != '0.0,0.0']

#list of longitude and latitude from bike station dataframe
bike_ll = list(set(data['ll']))

In [ ]:
#replace , so we can add the ll of stations to URL
bike_stop_ll = [s.replace(',', '%2C') for s in bike_ll]

In [ ]:
# went to https://location.foursquare.com/developer/reference/place-search and 
# https://location.foursquare.com/developer/reference/response-fields to check how API call should be made
# checked categories https://location.foursquare.com/places/docs/categories

#set the key
api_key = os.environ["FOURSQUARE_KEY"]

# Create dictionary for headers
headers = {"Accept": "application/json"}

# Add key with our API KEY
headers['Authorization'] = api_key

In [ ]:
foursquare_list_rich_20 = []

# Getting data for places around bike stops
for station in bike_stop_ll:
    try:
        url = "https://api.foursquare.com/v3/places/search?radius=1000&fields=categories%2Crating%2Cgeocodes%2Ccategories%2Clocation%2Cname%2Cfsq_id%2Cstats%2Cprice&limit=20&ll=" + station
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            foursquare_list_rich_20.append(response.json())
        else:
            print('Error occurred during the API request')
    except Exception as e:
        print(e)
        if e == "Quota exceeded":
            print("Exceeded quota: waiting for an hour")
            time.sleep(3600)


In [ ]:
df_foursquare_20 = pd.DataFrame(columns=['row_from_station', 'fsq_id', 'category_id', 'name', 'latitude', 'longitude', 'rating', 'total_ratings', 'total_photos', 'total_tips'])

for station in range(len(foursquare_list_rich_20)):
    for result in range(len(foursquare_list_rich_20[station]['results'])):
        if 'fsq_id' in foursquare_list_rich_20[station]['results'][result]:
            fsq_id = foursquare_list_rich_20[station]['results'][result]['fsq_id']
        else:
            fsq_id = None
            
        if 'categories' in foursquare_list_rich_20[station]['results'][result] and len(foursquare_list_rich_20[station]['results'][result]['categories']) > 0:
            category_id = foursquare_list_rich_20[station]['results'][result]['categories'][0].get('id', None)
        else:
            category_id = None
            
        if 'name' in foursquare_list_rich_20[station]['results'][result]:
            name = foursquare_list_rich_20[station]['results'][result]['name']
        else:
            name = None
            
        if 'geocodes' in foursquare_list_rich_20[station]['results'][result] and 'main' in foursquare_list_rich_20[station]['results'][result]['geocodes']:
            latitude = foursquare_list_rich_20[station]['results'][result]['geocodes']['main'].get('latitude', None)
            longitude = foursquare_list_rich_20[station]['results'][result]['geocodes']['main'].get('longitude', None)
        else:
            latitude = None
            longitude = None
            
        if 'rating' in foursquare_list_rich_20[station]['results'][result]:
            rating = foursquare_list_rich_20[station]['results'][result]['rating']
        else:
            rating = None
            
        if 'stats' in foursquare_list_rich_20[station]['results'][result]:
            total_ratings = foursquare_list_rich_20[station]['results'][result]['stats'].get('total_ratings', None)
            total_photos = foursquare_list_rich_20[station]['results'][result]['stats'].get('total_photos', None)
            total_tips = foursquare_list_rich_20[station]['results'][result]['stats'].get('total_tips', None)
        else:
            total_ratings = None
            total_photos = None
            total_tips = None
        
        df_foursquare_20 = df_foursquare_20.append({
            'row_from_station': station,
            'fsq_id': fsq_id,
            'category_id': category_id,
            'name': name,
            'latitude': latitude,
            'longitude': longitude,
            'rating': rating,
            'total_ratings': total_ratings,
            'total_photos': total_photos,
            'total_tips': total_tips
        }, ignore_index=True)


In [ ]:
#foursquare_list (foursquare_basic.js) is a file that has only the basic information about business

#foursquare_list_rich_50 (foursquare_50.js) is file that has the same code block as foursquare_rich_20, except limit for number of businesses was 50
# df_foursquare_50.csv is dataframe for same file

In [ ]:
# I already made DataFrame when parsing through foursquare json file

df_foursquare_20.to_csv('df_foursquare_20.csv', index=False)

In [ ]:
#visited https://docs.developer.yelp.com/reference/v3_business_search to check API structure

#set the key
yelp_api_key = os.environ["YELP_KEY"]

# Create dictionary for headers
headers = {"Accept": "application/json"}

# Add key with our API KEY
headers['Authorization'] = yelp_api_key

headers = {
    "accept": "application/json",
    "Authorization": "Bearer " + yelp_api_key

In [ ]:
yelp_list = []

# Getting data for places around bike stops
for index in range(len(bike_stop_ll)):
    try:
        latitude = data.iloc[index]["latitude"]
        longitude = data.iloc[index]["longitude"]
        yelp_url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(latitude) + "&longitude=" + str(longitude) + "&term=park&radius=1000&categories=&sort_by=best_match&limit=20"
        yelp_response = requests.get(yelp_url, headers=headers)
        
        if yelp_response.status_code == 200:
            yelp_list.append(yelp_response.json())
        else:
            print('Error occurred during the API request')
    except Exception as e:
        print(e)
        if e == "Quota exceeded":
            print("Exceeded quota: waiting for an hour")
            time.sleep(3600)

# Save yelp_list as a JavaScript file
file_path = "yelp_list.js"

with open(file_path, "w") as js_file:
    js_file.write("var yelpList = ")
    json.dump(yelp_list, js_file)

print("yelp_list saved as js file: " + file_path)


In [ ]:
df_yelp = pd.DataFrame(columns=['row_from_station', 'id', 'name', 'rating', 'review_count', 'latitude', 'longitude'])

In [ ]:
#parsing through 'yelp_list' from previous step to get columns for dataframe (id, name, rating, review_count, latitude and longitude)
for station in range(len(yelp_list)):
    for business in range(len(yelp_list[station]['businesses'])):
        df_yelp = df_yelp.append({
            'row_from_station': station,
            'id': yelp_list[station]['businesses'][business]['id'],
            'name': yelp_list[station]['businesses'][business]['name'],
            'rating': yelp_list[station]['businesses'][business]['rating'],
            'review_count': yelp_list[station]['businesses'][business]['review_count'],
            'latitude': yelp_list[station]['businesses'][business]['coordinates']['latitude'],
            'longitude': yelp_list[station]['businesses'][business]['coordinates']['longitude']
            }, ignore_index=True)

In [ ]:
#checking shape and first 5 rows of dataframe
print(df_yelp.shape)
print(df_yelp.head())

In [ ]:
#remove duplicate rows
df_foursquare_20_clean = df_foursquare_20.drop_duplicates(subset=["fsq_id"], keep='first')
df_yelp_clean = df_yelp.drop_duplicates(subset=["id"], keep='first')

In [ ]:
df_foursquare_filter = df_foursquare_20_clean[
    (df_foursquare_20_clean.apply(lambda row: row.astype(str).str.contains('Parc', case=False).any(), axis=1)) &
    (df_foursquare_20_clean['total_ratings'] > 10)
]
print('Mean for total ratings is: ' + str(df_foursquare_filter['total_ratings'].mean()))
df_foursquare_filter.sort_values(by="rating", ascending=False).head(10)

In [ ]:
df_yelp_clean_filter = df_yelp_clean.query('review_count > 10')
print("Mean for review_count is: " + str(df_yelp_clean_filter['review_count'].mean()))
df_yelp_clean.query('review_count > 10').sort_values(by="rating", ascending=False).head(10)